# 1. Instalação da biblioteca do Google Generative AI

In [ ]:
!pip install -q -U pydantic-ai google-genai pandas nest_asyncio

import pandas as pd
import os
import nest_asyncio
import asyncio
from pydantic import BaseModel, Field
from typing import List, Optional
from pydantic_ai import Agent
from google.colab import userdata

# Aplica patch para permitir loops assíncronos no Colab/Jupyter
nest_asyncio.apply()

# Configuração

In [ ]:
class IndicadorNivel5(BaseModel):
    instituicao_fonte: str = Field(description="O instituto ou órgão oficial que gerou o dado (ex: IBGE, TSE, Ministério da Saúde).")
    indicador_social: str = Field(description="O nome do indicador ou estatística (ex: 'Taxa de desemprego', 'Índice de dengue', 'Gasto público').")
    valor_citado: str = Field(description="O número exato divulgado na matéria (ex: '15%', 'R$ 2 milhões', '30 mil casos').")

class ClassificacaoMancini(BaseModel):
    # Nível 5 é a base: se tem dado oficial, é True.
    enquadra_nivel_5_ou_superior: bool = Field(description="True se a matéria cita indicadores sociais/econômicos de institutos, mesmo que superficialmente.")
    elementos_nivel_5: List[IndicadorNivel5]

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


In [ ]:
# O 'pydantic-ai' usa providers. Para Gemini, geralmente usamos o prefixo 'google-gla:' ou 'gemini:'
# Se der erro no nome do modelo, tente 'google-genai:gemini-1.5-flash'
agent = Agent(
    'google-gla:gemini-2.5-flash-lite',
    output_type=ClassificacaoMancini,
    system_prompt=(
        "Você é um pesquisador classificando notícias segundo a matriz de Mancini & Vasconcellos (2016)."
        "\n\n"
        "### SEU ALVO: NÍVEL 5 (Jornalismo COM Dados) e superiores."
        "\nSegundo a definição, você DEVE EXTRAIR notícias que contenham:"
        "\n1. **Divulgação de Indicadores:** Dados econômicos, sociais ou financeiros repassados por institutos (IBGE, IPEA, TSE, Secretarias de Saúde/Segurança)."
        "\n2. **Reprodução de Balanços:** Matérias curtas que apenas citam um dado ('A inflação foi de X%', 'Dengue aumentou Y%')[cite: 465]."
        "\n3. **Uso Tangencial:** O dado é usado apenas para ilustrar um contexto (ex: uma matéria sobre favelas que cita o número de habitantes)."
        "\n\n"
        "### O QUE NÃO É NÍVEL 5 (IGNORE):"
        "\n- **Fatos Isolados (Não-Indicadores):** 'Homem levou 3 tiros', 'Carro bateu a 80km/h'. Isso NÃO é indicador social/econômico."
        "\n- **Opinião Pura:** Sem base numérica de instituto."
        "\n\n"
        "### INSTRUÇÃO:"
        "\nSe a notícia disser 'Segundo o IBGE, o PIB cresceu 1%', ISSO É NÍVEL 5. Extraia."
        "\nSe a notícia disser 'A Polícia registrou 50 roubos no mês', ISSO É NÍVEL 5 (Indicador de Segurança). Extraia."
    )
)

# Processamento de arquivos

In [ ]:
async def processar_nivel_5():
    arquivos = [
        "Noticias de goiás - g1_noticias.csv",
        "Noticias de goiás - jornal_opcao_noticias.csv",
        "Noticias de goiás - lab_noticias.csv"
    ]

    for arquivo in arquivos:
        if not os.path.exists(arquivo):
            continue

        print(f"\n📑 Processando (Critério Nível 5 - Mancini): {arquivo}...")
        df = pd.read_csv(arquivo)

        col_eh_nivel5 = []
        col_fontes = []
        col_indicadores = []
        col_valores = []

        total = len(df)

        for index, row in df.iterrows():
            texto = str(row.get('text', ''))

            if index % 10 == 0: print(f"   Analisando {index}/{total}...")

            if len(texto) < 50:
                col_eh_nivel5.append(False)
                col_fontes.append("")
                col_indicadores.append("")
                col_valores.append("")
                continue

            try:
                # Prompt reforçando a inclusão do Nível 5
                prompt_input = (
                    f"Analise este texto. Ele se enquadra no Nível 5 de Mancini (cita indicadores de institutos)? "
                    f"Texto: {texto}"
                )

                result = await agent.run(prompt_input)
                dados = result.output

                if dados.enquadra_nivel_5_ou_superior and dados.elementos_nivel_5:
                    col_eh_nivel5.append(True)

                    fontes = " | ".join([e.instituicao_fonte for e in dados.elementos_nivel_5])
                    inds = " | ".join([e.indicador_social for e in dados.elementos_nivel_5])
                    vals = " | ".join([e.valor_citado for e in dados.elementos_nivel_5])

                    col_fontes.append(fontes)
                    col_indicadores.append(inds)
                    col_valores.append(vals)
                else:
                    col_eh_nivel5.append(False)
                    col_fontes.append("")
                    col_indicadores.append("")
                    col_valores.append("")

            except Exception as e:
                col_eh_nivel5.append(False)
                col_fontes.append("ERRO")
                col_indicadores.append("")
                col_valores.append("")

            await asyncio.sleep(1.0)

        # Salva
        df['mancini_nivel_5'] = col_eh_nivel5
        df['mancini_instituicoes'] = col_fontes
        df['mancini_indicadores'] = col_indicadores
        df['mancini_valores'] = col_valores

        nome_saida = arquivo.replace(".csv", "_nivel5.csv")
        df.to_csv(nome_saida, index=False)
        print(f"✅ Salvo: {nome_saida}")

# Executa
asyncio.run(processar_nivel_5())


📑 Processando (Critério Nível 5 - Mancini): Noticias de goiás - g1_noticias.csv...
   Analisando 0/204...
   Analisando 10/204...
   Analisando 20/204...
   Analisando 30/204...
   Analisando 40/204...
   Analisando 50/204...
   Analisando 60/204...
   Analisando 70/204...
   Analisando 80/204...
   Analisando 90/204...
   Analisando 100/204...
   Analisando 110/204...
   Analisando 120/204...
   Analisando 130/204...
   Analisando 140/204...
   Analisando 150/204...
   Analisando 160/204...
   Analisando 170/204...
   Analisando 180/204...
   Analisando 190/204...
   Analisando 200/204...
✅ Salvo: Noticias de goiás - g1_noticias_nivel5.csv

📑 Processando (Critério Nível 5 - Mancini): Noticias de goiás - jornal_opcao_noticias.csv...
   Analisando 0/264...
   Analisando 10/264...
   Analisando 20/264...
   Analisando 30/264...
   Analisando 40/264...
   Analisando 50/264...
   Analisando 60/264...
   Analisando 70/264...
   Analisando 80/264...
   Analisando 90/264...
   Analisando 